In [217]:
import os
import pandas as pd
import datetime
import numpy as np

In [218]:
# Reading data and adding test and train data together:

df = pd.read_csv("Road_Traffic_Fine_Management_Process-test.csv")
df = df.append(pd.read_csv('Road_Traffic_Fine_Management_Process-training.csv'))

In [219]:
# Show the unique types of events.
df['event concept:name'].unique()

array(['Create Fine', 'Payment', 'Send Fine', 'Insert Fine Notification',
       'Insert Date Appeal to Prefecture', 'Send Appeal to Prefecture',
       'Add penalty', 'Receive Result Appeal from Prefecture',
       'Notify Result Appeal to Offender', 'Appeal to Judge',
       'Send for Credit Collection'], dtype=object)

In [220]:
# Convert the time column to a python datetime object:

def to_date(x):
    day, month, year = str(x).split(" ")[0].split('-')
    return datetime.date(int(year), int(month), int(day))


df['event time:timestamp'] = df['event time:timestamp'].apply(to_date)

In [221]:
df = df.sort_values(by=['case concept:name', 'event time:timestamp'], ascending=[True, False])
case = None
inter_event_times = []
total_times = []
end = None
for index, row in df.iterrows():
    current = row['case concept:name']
    if case != current:
        if end != None:
            total_times.append(end-start)
        case = current
        start = row['event time:timestamp']
        inter_event_times.append(datetime.timedelta(0))
    else:
        end = row['event time:timestamp']
        inter_event_times.append(start-end)
df['time-to-end'] = inter_event_times

df = df.sort_values(by=['case concept:name', 'event time:timestamp'])
case = None
inter_event_times = []
total_times = []
end = None
for index, row in df.iterrows():
    current = row['case concept:name']
    if case != current:
        if end != None:
            total_times.append(end-start)
        case = current
        start = row['event time:timestamp']
        inter_event_times.append(datetime.timedelta(0))
    else:
        end = row['event time:timestamp']
        inter_event_times.append(end-start)
        
df['inter-event-time'] = inter_event_times

In [225]:
mean = sum(i.total_seconds() for i in total_times)/len(total_times)
#estimated = 
def to_seconds(x):
    return x.total_seconds()
df['estimator'] = mean - df['inter-event-time'].apply(to_seconds) 


In [226]:
def to_days(x):
    return datetime.timedelta(seconds=x)
df['estimator'] = df['estimator'].apply(to_days)

In [241]:
sum((df['time-to-end'] - df['estimator']).apply(to_seconds).apply(lambda x: x ** 2))/len(df)

978045278038150.4

In [242]:
df

,eventID,case concept:name,event concept:name,event lifecycle:transition,event time:timestamp,time-to-end,inter-event-time,estimator
188244,0,A1,Create Fine,complete,2006-07-24,134 days,0 days,341 days 16:04:35.801528
206285,1,A1,Send Fine,complete,2006-12-05,0 days,134 days,207 days 16:04:35.801528
189251,4294967296,A100,Create Fine,complete,2006-08-02,971 days,0 days,341 days 16:04:35.801528
207056,4294967297,A100,Send Fine,complete,2006-12-12,839 days,132 days,209 days 16:04:35.801528
213793,4294967298,A100,Insert Fine Notification,complete,2007-01-15,805 days,166 days,175 days 16:04:35.801528
224272,4294967299,A100,Add penalty,complete,2007-03-16,745 days,226 days,115 days 16:04:35.801528
303645,4294967300,A100,Send for Credit Collection,complete,2009-03-30,0 days,971 days,-630 days +16:04:35.801528
223558,8589934592,A10000,Create Fine,complete,2007-03-09,550 days,0 days,341 days 16:04:35.801528
235145,8589934593,A10000,Send Fine,complete,2007-07-17,420 days,130 days,211 days 16:04:35.801528
237213,8589934594,A10000,Insert Fine Notification,complete,2007-08-02,404 days,146 days,195 days 16:04:35.801528


In [25]:
# Filter out the cases that seem to have no sensible end.

cases = df[(df['event concept:name'] == 'Send Appeal to Prefecture') | (df['event concept:name'] == 'Payment') | (df['event concept:name'] == 'Send for Credit Collection')]['case concept:name']
special_cases = np.setdiff1d(df['case concept:name'].unique(), np.array(cases))
df[df['case concept:name'].isin(special_cases)].sort_values(by=['case concept:name', 'event time:timestamp'])

,eventID,case concept:name,event concept:name,event lifecycle:transition,event time:timestamp
2742,516886429171712,S51241,Create Fine,complete,2000-06-24
8118,516886429171713,S51241,Send Fine,complete,2000-08-24
2161,516980918452224,S51278,Create Fine,complete,2000-06-16
7725,516980918452225,S51278,Send Fine,complete,2000-08-24
2345,516989508386816,S51282,Create Fine,complete,2000-06-18
7726,516989508386817,S51282,Send Fine,complete,2000-08-24
2825,517023868125184,S51294,Create Fine,complete,2000-06-25
7732,517023868125185,S51294,Send Fine,complete,2000-08-24
4312,517131242307584,S51338,Create Fine,complete,2000-07-15
9361,517131242307585,S51338,Send Fine,complete,2000-08-29
